### импортируем все нужное

In [1]:
import re
import pandas as pd
import csv
from tqdm import tqdm

In [2]:
try:
    from redditscore.tokenizer_rus import CrazyTokenizer as RusCrazyTokenizer
except:
    print('положите в папку библы redditscore файл tokenizer_rus.py (рядом с обычным tokenizer.py)')

from redditscore.tokenizer import CrazyTokenizer as EngCrazyTokenizer

In [21]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [22]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

## Расширяем список стопслов

In [10]:
from nltk.corpus import stopwords

In [11]:
stops = stopwords.words('russian')
stops.extend(stopwords.words('english'))

In [12]:
badwords = [
'я', 'а', 'да', 'но', 'тебе', 'мне', 'ты', 'и', 'у', 'на', 'ага',
'так', 'там', 'какие', 'который', 'какая', 'туда', 'давай', 'короче', 'кажется', 'вообще',
'ну', 'чет', 'неа', 'свои', 'наше', 'хотя', 'такое', 'например', 'кароч', 'как-то',
'нам', 'хм', 'всем', 'нет', 'да', 'оно', 'своем', 'про', 'вы', 'тд', 'тп', 'т.д', 'т.п',
'вся', 'вам', 'это', 'эта', 'эти', 'этот', 'прям', 'либо', 'как', 'мы',
'просто', 'блин', 'очень', 'самые', 'твоем', 'ваша', 'кстати', 'вроде', 'типа', 'пока', 'ок',
    '°', '͜ʖ', '͡', 'ツ', 'ʕ•ᴥ•ʔ'
]

In [13]:
stops.extend(badwords)
stops.remove('не')
stops.remove('нет')
stops.remove('есть')

In [14]:
print(stops)

['и', 'в', 'во', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'сво

## Расширяем список мер

чтоб не лезть руками в код спейси делаем чуть более костыльно

In [15]:
from spacy.lang.char_classes import UNITS
units = UNITS.split('|')
for u in 'кг|г|мг|мкг|л|мл|гр'.split('|'):
    if u not in units:
        units.append(u)
        
print(units)

['km', 'km²', 'km³', 'm', 'm²', 'm³', 'dm', 'dm²', 'dm³', 'cm', 'cm²', 'cm³', 'mm', 'mm²', 'mm³', 'ha', 'µm', 'nm', 'yd', 'in', 'ft', 'kg', 'g', 'mg', 'µg', 't', 'lb', 'oz', 'm/s', 'km/h', 'kmh', 'mph', 'hPa', 'Pa', 'mbar', 'mb', 'MB', 'kb', 'KB', 'gb', 'GB', 'tb', 'TB', 'T', 'G', 'M', 'K', '%', 'км', 'км²', 'км³', 'м²', 'м³', 'дм', 'дм²', 'дм³', 'см', 'см²', 'см³', 'мм', 'мм²', 'мм³', 'нм', 'кг', 'г', 'мг', 'м/с', 'км/ч', 'кПа', 'Па', 'мбар', 'Кб', 'КБ', 'кб', 'Мб', 'МБ', 'мб', 'Гб', 'ГБ', 'гб', 'Тб', 'ТБ', 'тбكم', 'كم²', 'كم³', 'م', 'م²', 'م³', 'سم', 'سم²', 'سم³', 'مم', 'مم²', 'مم³', 'كم', 'غرام', 'جرام', 'جم', 'كغ', 'ملغ', 'كوب', 'اكواب', 'мкг', 'л', 'мл', 'гр']


### Открываем данные

In [26]:
with open('../Data/task2_ru_training.tsv', 'r', encoding='utf-8') as f:
    df = pd.read_csv(f, sep="\t")

In [27]:
df.head(5)

,tweet_id,tweet,class
0,760402871867367424,"Настало время для ингаляторов. Дружок, Сальбут...",0
1,1035908416869462016,15) На прошлой зимней олимпиаде большинство лы...,1
2,1089839736427032577,Не соглашусь с заменой ЗОК на метопролол в так...,0
3,779671488748224513,"@di2m1 мезим Смекта Если отравление, то лоперамид",0
4,738309299756240897,Уберите микроволновки и имодиум Действуют соу...,0


### выбираем тестовые твиты

In [28]:
test_data = [df['tweet'][8],
             df['tweet'][9], # дефисы
             df['tweet'][19],
             df['tweet'][268], # дефисы, капс
             
             df['tweet'][273], # латиница, нет пробелов вокруг скобки
             
             df['tweet'][10], # 0,5г ; р/д
             df['tweet'][20], # 125 мгк 2 раза ; Вентолин 100 мгк
            
             df['tweet'][2368], # нет пробела между предложениями
             
             df['tweet'][14], # кавычки, юзернейм, эмоджи
             df['tweet'][18], # хештеги, юзернейм
             df['tweet'][25], # смайл, юзернейм, эмоджи
             df['tweet'][116], # эмоджи
             df['tweet'][270], # смайл
             
             df['tweet'][232] # сокращенная ссылка
            ]

In [13]:
#for tweet in test_data:
#    print(tweet)

## Настраиваем токенизатор

тут версия на улучшенном русском спейси [отсюда](https://github.com/aatimofeev/spacy_russian_tokenizer)

In [16]:
pos_emojis=['😀', '😁', '😂', '🤣', '😃', '😄', '😆', '😊', '😋', '😎', '😍', '😘', '🥰', '😚', '☺️', '🙂', '🤗', '🤩', '😏', '😌', '😛', '😜', '😝', '🙃', '🤑', '🤠', '😇', '🥳', '😈', '😺', '😸', '😹', '😻', '😽', '👍', '✌️', '🤟', '🤘', '👌', ' 🤙', '❤️', '🧡', '💛', '💚', '💙', '💜', '🖤', '❣️', '💕', '💞', '💓', '💗', '💖', '💘', '💝', '💟', '♥️', '🎊', '🎉', '🔝', '✔️', '🆒','☑️', '💑', '👩‍❤️‍👩', '👨‍❤️‍👨', '💏', '👩‍❤️‍💋‍👩', '👨‍❤️‍💋‍👨', '☀️', '🎆', '🌅', '🌄', '🌠', '🎇', '🎆', '🌇', '🌌', '💡', '🏖', '💯', '✅', '🎁', '💪', '👼']
neg_emojis=['🤨', '😐', '😑', '😶', '🙄', '😣', '😥', '🤐', '😫', '😓', '😔', '😕', '☹️', '🙁', '😖', '😞', '😟', '😤', '😢', '😭', '😦', '😧', '😨', '😩', '🤯', '😬', '😰', '😱', '😡', '😠', '🤬', '🤥', '👿', '😾', '😿', '🙀', '👎', '🤦‍♀️', '🤦‍♂️', '🙎‍♀️', '🙎‍♂️', '🙍‍♀️', '🙍‍♂️', '🥀', '🌥', '🌧', '⛈', '💔', '💢', '🔴', '🚨', '🧯', '⁉️']
neutral_emojis=['🤤', '😴', '🤒', '🌡️', '🤢', '😪', '🤮', '🤕', '💉', '💊', '😷', '😵', '💫', '🚑', '🦠', '🏥', '🤧', '🩺', '👨‍⚕️', '👩‍⚕️', '⚕️', '🧪', '🥵',  '🥶', '🥴', '💩', '🧟‍♀️', '🦴', '🦷', '🚽', '🦻', '👨‍🦯', '👩‍🦯', '👨‍🦼', '👩‍🦼', '👨‍🦽', ' 👩‍🦽', '🦼', '🦽', '🦯', '🩹', '♿️', '🚻', '🧫', '🧟‍♂️', '💤']


In [24]:
tokenizer_rus = RusCrazyTokenizer(lowercase=True, # к нижнему регистру
                           keepcaps=True, # все кроме all капс
                           urls='ЮРЛ', # удаляем ссылки
                           twitter_handles='ЮЗЕРНЕЙМ', # заменяем все юзернеймы
                           hashtags='ХЕШТЕГ', # пытаемся делить хештеги на слова (тут надо свой частотник закинуть)
                           pos_emojis=pos_emojis, neg_emojis=neg_emojis, neutral_emojis=neutral_emojis,
                           normalize=3, # удаляем больше 3 одинаковых знаков подряд
                           remove_punct=True, # удаляем пунктуацию
                           remove_breaks=True, # удаляем переносы строк
                           ignore_stopwords=stops, # удаляет стоп слова (разобраться почему падает на кастомном списке)
                           numbers = False,
                           extra_patterns = False
                          )

это на оригинальном английском (если не запускается русская)

In [24]:
tokenizer_eng = EngCrazyTokenizer(lowercase=True, # к нижнему регистру
                           keepcaps=True, # все кроме all капс
                           urls='ЮРЛ', # удаляем ссылки
                           twitter_handles='ЮЗЕРНЕЙМ', # заменяем все юзернеймы
                           hashtags='ХЕШТЕГ', # пытаемся делить хештеги на слова (тут надо свой частотник закинуть)
                           pos_emojis=True, neg_emojis=True, neutral_emojis=True, # меняем эмоджи на плейсхолдеры (кастомизировать)
                           normalize=3, # удаляем больше 3 одинаковых знаков подряд
                           remove_punct=True, # удаляем пунктуацию
                           remove_breaks=True, # удаляем переносы строк
                           ignore_stopwords=stops, # удаляет стоп слова (разобраться почему падает на кастомном списке)
                           numbers = False,
                           extra_patterns = False
                          )

In [25]:
#vocab_rus = list(tokenizer_rus._nlp.vocab.strings)
#vocab_eng = list(tokenizer_eng._nlp.vocab.strings)

## Добавляем еще несколько удалений и замен

In [32]:
def process(tweet, log = False, lang = 'rus') -> str:
    tweet = re.sub('(\w)\.([А-ЯЁ])', r'\1. \2', tweet) # ставим пробелы
    tweet = re.sub('-', ' ', tweet) # меняем деффисы на пробел
    if log:
        #print('!до токенизации')
        print(tweet)
    # токенизируем (см. удаления и замены выше)
    if lang == 'rus':
        tokens = tokenizer_rus.tokenize(tweet)
    elif lang == 'eng':
        tokens = tokenizer_eng.tokenize(tweet)
    else:
        raise KeyError('ты долбаеб? языка {} нет'.format(lang))
        
    if log:
        #print('!после токенизации')
        print(tokens)
        
    new_tokens = []
    for token in tokens:
        if re.search('\d+', token):
            try:
                x = re.findall('[^\d.,\s]+', token)[0]
                if x in units:
                    token = 'NUM MEASURE'
                else:
                    continue
            except:
                token = 'NUM'
        if re.search('[!"#$%&\'()*+/:;<=>?@[\]^_`{|}~]', token): #удаляем оставшиеся символы
            token = re.sub('[!"#$%&\'()*+/:;<=>?@[\]^_`{|}~]', '', token)
        
        if token in units:
            token = 'MEASURE'
        
        if len(token) < 2: # and not re.search('[,\.!"#$%&\'()*+/:;<=>?@[\]^_`{|}~]', token):
            continue
            
        new_tokens.append(token)
            
    if log:
        #print('!после обработки каждого токена')
        print(new_tokens)
        #print('!output')
    
    return ' '.join(new_tokens)

In [33]:
res = []
for tweet in test_data:
    print(tweet)
    tweet = process(tweet, log=False, lang='rus')
    res.append(tweet)
    print(tweet, '\n')

Все эти слюнтяи, утверждающие, будто суицид-удел слабых людей, найдут тысячу причин не глотать горсть прозака.Им просто не хватает смелости.
слюнтяи , утверждающие , суицид удел слабых людей , найдут тысячу причин не глотать горсть прозака . не хватает смелости . 

Это вам не лыжи,сальбутамол не поможет. Россия - Норвегия 5:1
не лыжи , сальбутамол не поможет . россия норвегия 

Если оланзапин мне не поможет, тогда мне назначат галоперидол. Не хочу докатываться до галоперидола. Надо что-то делать с этой проклятой тревожностью.
оланзапин не поможет , назначат галоперидол . не хочу докатываться галоперидола . делать проклятой тревожностью . 

из-за флуоксетина катастрофически не хочется что-либо есть, но почему-то БЛИНЫ ИЗ ТЕРЕМКА в меня влезают всегда  Теремок, спасибо что живая
флуоксетина катастрофически не хочется есть , почему БЛИНЫ ТЕРЕМКА влезают теремок , спасибо живая 

В моей голове Хенвон любит lil peep; перкосет, молли, перпл дранк(это все наркота, если кто не в курсе); органи

In [18]:
res

['слюнтяи утверждающие суицид удел слабых людей найдут тысячу причин не глотать горсть прозака не хватает смелости',
 'не лыжи сальбутамол не поможет россия норвегия',
 'оланзапин не поможет назначат галоперидол не хочу докатываться галоперидола делать проклятой тревожностью',
 'флуоксетина катастрофически не хочется есть почему БЛИНЫ ТЕРЕМКА влезают теремок спасибо живая',
 'моей голове хенвон любит lil peep перкосет молли перпл дранк наркота не курсе организует нелегальные вечеринки',
 'ципрофлоксацин NUM NUM MEASURE NUM смысл жизни',
 'ветеринар прописал ингаляции фликсотидом NUM мгк NUM раза сутки прописал вентолин NUM мгк случай приступов',
 'день выпила NUM таблеток алпразолама чувствую расслабленно ох днем часа NUM спала',
 'ЮЗЕРНЕЙМ не лечатся футболисты астмы синдрома дефицита внимания столь любимыми нероссийских спортсменов законными амфетамином декстроамфетамином NEGEMOJI',
 'ЮЗЕРНЕЙМ порекомендовал сотрудникам wada выпить мельдония вдохнуть сальбутамола попробовать приблизи

## тут можно тестировать конкретные случаи и писать о  траблах ане

In [29]:
test_phrase = 'я вдолбил 100г мефа и 20мл пива 50л колы и 18кг моркови а еще потом занюхал 18мг герыча'
log = False
lang = 'rus'

In [30]:
test_tokenizer = EngCrazyTokenizer(lowercase=True, # к нижнему регистру
                           keepcaps=True, # все кроме all капс
                           urls='ЮРЛ', # удаляем ссылки
                           twitter_handles='ЮЗЕРНЕЙМ', # заменяем все юзернеймы
                           hashtags='ХЕШТЕГ', # пытаемся делить хештеги на слова (тут надо свой частотник закинуть)
                           pos_emojis=False, neg_emojis=False, neutral_emojis=False, # меняем эмоджи на плейсхолдеры (кастомизировать)
                           normalize=3, # удаляем больше 3 одинаковых знаков подряд
                           remove_punct=True, # удаляем пунктуацию
                           remove_breaks=True, # удаляем переносы строк
                           ignore_stopwords=False, # удаляет стоп слова (разобраться почему падает на кастомном списке)
                           numbers = False,
                           extra_patterns = False
                          )

In [31]:
print('before  :', test_phrase)
res_phrase = process(test_phrase, log = log, lang = lang)
print('after   :',res_phrase)

print('del/repl:', ', '.join([w for w in test_tokenizer.tokenize(test_phrase) if w not in res_phrase.split(' ')]))

before  : я вдолбил 100г мефа и 20мл пива 50л колы и 18кг моркови а еще потом занюхал 18мг герыча
after   : вдолбил NUM MEASURE мефа NUM MEASURE пива NUM MEASURE колы NUM MEASURE моркови занюхал NUM MEASURE герыча
del/repl: я, 100, г, и, 20мл, 50л, и, 18, кг, а, еще, потом, 18, мг


## Лемматизация

### nltk snowball stemmer

In [33]:
for tweet in res:
    print(tweet)
    print(' '.join([stemmer.stem(w) for w in tweet.split(' ')]), '\n')

слюнтяи утверждающие суицид удел слабых людей найдут тысячу причин не глотать горсть прозака не хватает смелости
слюнтя утвержда суицид удел слаб люд найдут тысяч причин не глота горст прозак не хвата смелост 

не лыжи сальбутамол не поможет россия норвегия
не лыж сальбутамол не поможет росс норвег 

оланзапин не поможет назначат галоперидол не хочу докатываться галоперидола делать проклятой тревожностью
оланзапин не поможет назначат галоперидол не хоч докатыва галоперидол дела проклят тревожн 

флуоксетина катастрофически не хочется есть почему БЛИНЫ ТЕРЕМКА влезают теремок спасибо живая
флуоксетин катастрофическ не хочет ест поч блин теремк влеза теремок спасиб жив 

моей голове хенвон любит lil peep перкосет молли перпл дранк наркота не курсе организует нелегальные вечеринки
мо голов хенвон люб lil peep перкосет молл перпл дранк наркот не курс организ нелегальн вечеринк 

ципрофлоксацин NUM NUM MEASURE NUM смысл жизни
ципрофлоксацин NUM NUM MEASURE NUM смысл жизн 

ветеринар прописа

### Mystem

In [19]:
from pymystem3 import Mystem
mstm = Mystem()

In [24]:
lem = []
for tweet in tqdm(res):
    lem.append(''.join(mstm.lemmatize(tweet)).strip())

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:13<00:00,  1.12it/s]


In [25]:
lem

['слюнтяй утверждать суицид удел слабый человек находить тысяча причина не глотать горсть прозак не хватать смелость',
 'не лыжа сальбутамол не помогать россия норвегия',
 'оланзапина не помогать назначать галоперидол не хотеть докатываться галоперидол делать проклятый тревожность',
 'флуоксетин катастрофически не хотеться есть почему блин теремок влезать теремок спасибо живой',
 'мой голова хенвон любить lil peep перкосать молль перпл дранк наркота не курс организовывать нелегальный вечеринка',
 'ципрофлоксацин NUM NUM MEASURE NUM смысл жизнь',
 'ветеринар прописывать ингаляция фликсотид NUM мгк NUM раз сутки прописывать вентолина NUM мгк случай приступ',
 'день выпивать NUM таблетка алпразолам чувствовать расслабленно ох день час NUM спать',
 'юзернейм не лечиться футболист астма синдром дефицит внимание столь любимый нероссийский спортсмен законный амфетамин декстроамфетамин NEGEMOJI',
 'юзернейм порекомендовать сотрудник wada выпивать мельдония вдыхать сальбутамол попробовать прибл

### pymorphy

In [21]:
lem2 = []
for tweet in tqdm(res):
    lem_tweet = []
    for word in tweet.split():
        lem_tweet.append(morph.parse(word)[0].normal_form)
    lem2.append(' '.join(lem_tweet))

100%|█████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 326.15it/s]


In [22]:
lem2

['слюнтяй утверждать суицид удел слабый человек найти тысяча причина не глотать горсть прозак не хватать смелость',
 'не лыжа сальбутамол не помочь россия норвегия',
 'оланзапина не помочь назначить галоперидол не хотеть докатываться галоперидол делать проклятый тревожность',
 'флуоксетина катастрофически не хотеться есть почему блин теремок влезать теремок спасибо живой',
 'мой голов хенвон любить lil peep перкосет молли перплый дранк наркот не курс организовать нелегальный вечеринка',
 'ципрофлоксацин num num measure num смысл жизнь',
 'ветеринар прописать ингаляция фликсотид num мгк num раз сутки прописать вентолин num мгк случай приступ',
 'день выпить num таблетка алпразолам чувствовать расслабленно ох днём час num спасть',
 'юзернейм не лечиться футболист астма синдром дефицит внимание столь любимый нероссийский спортсмен законный амфетамин декстроамфетамин negemoji',
 'юзернейм порекомендовать сотрудник wada выпить мельдония вдохнуть сальбутамол попробовать приблизиться результа

In [26]:
for t1, t2 in zip(lem, lem2):
    for w1, w2 in zip(t1.split(), t2.split()):
        if w1 != w2:
            print(w1, w2)

находить найти
помогать помочь
помогать помочь
назначать назначить
флуоксетин флуоксетина
голова голов
перкосать перкосет
молль молли
перпл перплый
наркота наркот
организовывать организовать
NUM num
NUM num
MEASURE measure
NUM num
прописывать прописать
NUM num
NUM num
прописывать прописать
вентолина вентолин
NUM num
выпивать выпить
NUM num
день днём
NUM num
спать спасть
NEGEMOJI negemoji
выпивать выпить
вдыхать вдохнуть
приближаться приблизиться
POSEMOJI posemoji
мема мем
еще ещё
сероквель сероквля
выясняться выясниться
четвертый четыре
обшаривать обшарить
кларитин кларитина
фликсоназ фликсоназа
паксил паксила
оправдывать оправдать
давать дать
паксить паксила
юрла юрл


## Соединение train и val

In [5]:
with open('../original_data/task2_ru_training.tsv', 'r', encoding='utf-8') as tr:
    with open('../original_data/task2_ru_validation.tsv', 'r', encoding='utf-8') as val:
        
        df1 = pd.read_csv(tr, sep="\t")
        df2 = pd.read_csv(val, sep="\t")

In [7]:
combined = pd.concat([df1, df2])

In [8]:
combined

,tweet_id,tweet,class
0,760402871867367424,"Настало время для ингаляторов. Дружок, Сальбут...",0
1,1035908416869462016,15) На прошлой зимней олимпиаде большинство лы...,1
2,1089839736427032577,Не соглашусь с заменой ЗОК на метопролол в так...,0
3,779671488748224513,"@di2m1 мезим Смекта Если отравление, то лоперамид",0
4,738309299756240897,Уберите микроволновки и имодиум Действуют соу...,0
5,1202310351943032834,@luvkismau это видео было снято во время прием...,0
6,933680693380571136,"Кроме того, есть выстраданные стандарты опреде...",0
7,738771297228951552,теперь я отчаянно пытаюсь найти тату с молекул...,0
8,688135039578550272,"Все эти слюнтяи, утверждающие, будто суицид-уд...",0
9,966241801706319878,"Это вам не лыжи,сальбутамол не поможет. Россия...",0


# Полный скрипт препроцессинга

In [34]:
def write_data(df, out_file, prep = 'tweet', lem = None, stem = None):
    
    if prep:
        new_tweets = []
        for tweet in tqdm(df[prep]):
            new_tweets.append(process(tweet, log=False, lang = 'rus'))
        df['prep_tweet'] = new_tweets
        
        
#     if lemmatize == 'mystem':
#         lem = []
#         for tweet in tqdm(df[prep_tweets]):
#             lem.append(''.join(mstm.lemmatize(tweet)).strip())
#         df['lem_tweet'] = lem

    if lem:
        lems = []
        for tweet in tqdm(df[lem]):
            lem_tweet = []
            for word in tweet.split():
                lem_tweet.append(morph.parse(word)[0].normal_form)
            lems.append(' '.join(lem_tweet))
        df['lem_tweet'] = lems
        
    
    if stem:
        stems = []
        for tweet in tqdm(df[stem]):
            stems.append(' '.join([stemmer.stem(w).strip() for w in tweet.split(' ')]))
        df['stem_tweet'] = stems


    with open('../Data/{}'.format(out_file), 'w', encoding='utf-8') as output:
        df.to_csv(output, sep='\t', index=False, line_terminator='\n')

    print('Done! Your data is in {}'.format(out_file))

In [35]:
out_file = 'combined_prep_lem_stem_withpunct.tsv'

In [36]:
write_data(combined, out_file,
           prep = 'tweet',
           lem = 'prep_tweet',
           stem = 'lem_tweet'
          )

100%|████████████████████████████████████████████████████████████████████████████| 7612/7612 [00:07<00:00, 1036.30it/s]


Done! Your data is in combined_prep_lem_stem_withpunct.tsv


In [39]:
pd.read_csv('../Data/{}'.format(out_file), sep='\t')

,tweet_id,tweet,class,prep_tweet,lem_tweet,stem_tweet
0,760402871867367424,"Настало время для ингаляторов. Дружок, Сальбут...",0,"настало время ингаляторов . дружок , сальбутам...","настать время ингалятор . дружок , сальбутамол...","наста врем ингалятор . дружок , сальбутамол , ?"
1,1035908416869462016,15) На прошлой зимней олимпиаде большинство лы...,1,NUM ) прошлой зимней олимпиаде большинство лыж...,num ) прошлый зимний олимпиада большинство лыж...,num ) прошл зимн олимпиад большинств лыжник пр...
2,1089839736427032577,Не соглашусь с заменой ЗОК на метопролол в так...,0,не соглашусь заменой ЗОК метопролол таком виде...,не согласиться замена зок метопролоть такой ви...,не соглас зам зок метопролот так вид . один фо...
3,779671488748224513,"@di2m1 мезим Смекта Если отравление, то лоперамид",0,"ЮЗЕРНЕЙМ мезим смекта отравление , лоперамид","юзернейм мезим смект отравление , лоперамид","юзернейм мез смект отравлен , лоперамид"
4,738309299756240897,Уберите микроволновки и имодиум Действуют соу...,0,уберите микроволновки имодиум действуют соулмэйты,убрать микроволновка имодиум действовать соулмэйт,убра микроволновк имодиум действова соулмэйт
5,1202310351943032834,@luvkismau это видео было снято во время прием...,0,ЮЗЕРНЕЙМ видео снято время приема оланзапина ?,юзернейм видео снятой время приём оланзапина ?,юзернейм виде снят врем при оланзапин ?
6,933680693380571136,"Кроме того, есть выстраданные стандарты опреде...",0,"кроме , есть выстраданные стандарты определени...","кроме , есть выстрадать стандарт определение э...","кром , ест выстрада стандарт определен эффекти..."
7,738771297228951552,теперь я отчаянно пытаюсь найти тату с молекул...,0,отчаянно пытаюсь найти тату молекулой флуоксетина,отчаянно пытаться найти тат молекула флуоксетина,отчая пыта найт тат молекул флуоксетин
8,688135039578550272,"Все эти слюнтяи, утверждающие, будто суицид-уд...",0,"слюнтяи , утверждающие , суицид удел слабых лю...","слюнтяй , утверждать , суицид удел слабый чело...","слюнтя , утвержда , суицид удел слаб человек ,..."
9,966241801706319878,"Это вам не лыжи,сальбутамол не поможет. Россия...",0,"не лыжи , сальбутамол не поможет . россия норв...","не лыжа , сальбутамол не помочь . россия норвегия","не лыж , сальбутамол не помоч . росс норвег"


In [40]:
pd.read_csv('../Data/{}'.format('combined_prep_lem_stem_nopunct.tsv'), sep='\t')

,tweet_id,tweet,class,prep_tweet,lem_tweet,stem_tweet
0,760402871867367424,"Настало время для ингаляторов. Дружок, Сальбут...",0,настало время ингаляторов дружок сальбутамол,настать время ингалятор дружок сальбутамол,наста врем ингалятор дружок сальбутамол
1,1035908416869462016,15) На прошлой зимней олимпиаде большинство лы...,1,NUM прошлой зимней олимпиаде большинство лыжни...,num прошлый зимний олимпиада большинство лыжни...,num прошл зимн олимпиад большинств лыжник прие...
2,1089839736427032577,Не соглашусь с заменой ЗОК на метопролол в так...,0,не соглашусь заменой ЗОК метопролол таком виде...,не согласиться замена зок метопролоть такой ви...,не соглас зам зок метопролот так вид один форм...
3,779671488748224513,"@di2m1 мезим Смекта Если отравление, то лоперамид",0,ЮЗЕРНЕЙМ мезим смекта отравление лоперамид,юзернейм мезим смект отравление лоперамид,юзернейм мез смект отравлен лоперамид
4,738309299756240897,Уберите микроволновки и имодиум Действуют соу...,0,уберите микроволновки имодиум действуют соулмэйты,убрать микроволновка имодиум действовать соулмэйт,убра микроволновк имодиум действова соулмэйт
5,1202310351943032834,@luvkismau это видео было снято во время прием...,0,ЮЗЕРНЕЙМ видео снято время приема оланзапина,юзернейм видео снятой время приём оланзапина,юзернейм виде снят врем при оланзапин
6,933680693380571136,"Кроме того, есть выстраданные стандарты опреде...",0,кроме есть выстраданные стандарты определения ...,кроме есть выстрадать стандарт определение эфф...,кром ест выстрада стандарт определен эффективн...
7,738771297228951552,теперь я отчаянно пытаюсь найти тату с молекул...,0,отчаянно пытаюсь найти тату молекулой флуоксетина,отчаянно пытаться найти тат молекула флуоксетина,отчая пыта найт тат молекул флуоксетин
8,688135039578550272,"Все эти слюнтяи, утверждающие, будто суицид-уд...",0,слюнтяи утверждающие суицид удел слабых людей ...,слюнтяй утверждать суицид удел слабый человек ...,слюнтя утвержда суицид удел слаб человек найт ...
9,966241801706319878,"Это вам не лыжи,сальбутамол не поможет. Россия...",0,не лыжи сальбутамол не поможет россия норвегия,не лыжа сальбутамол не помочь россия норвегия,не лыж сальбутамол не помоч росс норвег
